# Set up

In [10]:
%pip install rdflib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from rdflib import Graph, URIRef
import os

data_folder = "Data"
g = Graph()

for filename in os.listdir(data_folder):
    if filename.endswith(".ttl") or filename.endswith(".rdf") or filename.endswith(".txt"):
        file_path = os.path.join(data_folder, filename)
        #f = open(file_path)
        if filename != 'rdf_200_sphn.txt':
            g.parse(file_path, format="turtle")
print(f"Loaded {len(g)} triples.")

Loaded 0 triples.


In [12]:
predicates = set()
for _, p, _ in g.triples((None, None, None)):
    predicates.add(p)

for predicate in predicates:
    print(predicate)

In [13]:
subject_types = set()
for s, p, o in g.triples((None, None, None)):
    if p == URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'):
        subject_types.add(o)

for t in subject_types:
    print(t)

In [14]:
unique_predicates = set()
for _, predicate, _ in g.triples((None, None, None)):
    unique_predicates.add(predicate)

for pred in unique_predicates:
    print(pred)

In [15]:
object_types = set()
for _, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), None)):
    object_types.add(o)

for obj_type in object_types:
    print(obj_type)

In [16]:
example_tuples = []
predicate_uri = URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier')
for triple in g.triples((None, predicate_uri, None)):
    example_tuples.append(triple)
    if len(example_tuples) >= 5:
        break

for tup in example_tuples:
    print(tup)

# Star Shape Queries

## Question 1

In [17]:

qa_pairs = []
for s, p, o in g.triples((None, None, None)):
    if 'PATIENTS' in str(s):
        patient_id = str(s).split('/')[-1]
        if patient_id.isdigit():
            question = f"What medical procedures and diagnoses did patient {patient_id} have"
            # SPARQL query template (adjust predicates as needed)
            sparql_query = f"""
            SELECT ?MedicalProcedure ?Diagnosis WHERE {{
                ?MedicalProcedure <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?MedicalProcedure a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .
                ?Diagnosis <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?Diagnosis a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
            }}
            """
            qa_pairs.append((question, sparql_query.strip()))

#for q, a in qa_pairs:
#    print("Q:", q)
#    print("SPARQL:", a)
#    print()
len(qa_pairs)

0

In [18]:

#print(qa_pairs[1][1])
for row in g.query(qa_pairs[2][1]):
    print([str(item) for item in row])

IndexError: list index out of range

## Question 2

In [75]:
# Generate question and SPARQL query pairs for the new question template
new_qa_pairs = []
for s, p, o in g.triples((None, None, None)):
    if 'patient' in str(s) and 'DrugPrescription' in str(s): 
        if (str(s).split('/')[-1]).isdigit():
            patient_id = str(s).split('/')[-1]
            subject = "https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/" + patient_id
            question = f"What was the code of a test recorded for patient {patient_id}, who also had a Drug Prescription?"
            sparql_query = f"""
             SELECT ?Code WHERE {{
                ?DrugPrescription <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{subject}> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{subject}> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult .
                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .

             }}
             """
            new_qa_pairs.append((question, sparql_query.strip()))
len(new_qa_pairs)

66282

In [77]:
print(new_qa_pairs[1][1])
for row in g.query(new_qa_pairs[1][1]):
    print([str(item) for item in row])

SELECT ?Code WHERE {
                ?DrugPrescription <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/17095291> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/17095291> .
                ?LabTestEvent <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult .
                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .

             }
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/50893']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/50893']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/50893']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/50893']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/50893']
['https://www.biomedit.ch/rdf/sphn-schema/sphn/d_labitems/50893']
['https://www.biomedit.ch/rdf/

## Question type 3

In [88]:
# Generate question and SPARQL query pairs for the template:
# "Which patients, with the same lab results and the same diagnosis, have different Medical Procedures"

same_lab_diag_diff_proc_qa_pairs = []

# SPARQL query template
question = "Which patients, with the same lab results and the same diagnosis, have different Medical Procedures?"
sparql_query = """
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult1 .
    ?LabResult1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?ProcedureEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?ProcedureEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .
    
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult2 .
    ?LabResult2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?ProcedureEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?ProcedureEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .
    
    FILTER(?Patient1 != ?Patient2) .
    FILTER(?LabResult1 = ?LabResult2) .
    FILTER(?Procedure1 != ?Procedure2) .
}
"""

same_lab_diag_diff_proc_qa_pairs.append((question, sparql_query.strip()))
print(same_lab_diag_diff_proc_qa_pairs[0][0])
print(same_lab_diag_diff_proc_qa_pairs[0][1])

Which patients, with the same lab results and the same diagnosis, have different Medical Procedures?
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult1 .
    ?LabResult1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?ProcedureEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?ProcedureEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .

    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult2 .


In [89]:
for row in g.query(same_lab_diag_diff_proc_qa_pairs[0][1]):
    print([str(item) for item in row])

KeyboardInterrupt: 

## Question type 4

In [ ]:
active_ingredient_qa_pairs = []
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription'))):
    prescription_id = str(s).split('/')[-1]
    question = f"What active ingredients are in drug prescription {prescription_id}?"
    sparql_query = f"""
    SELECT ?ActiveIngredient WHERE {{
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
        ?Drug <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?ActiveIngredient .
    }}
    """
    active_ingredient_qa_pairs.append((question, sparql_query.strip()))

if active_ingredient_qa_pairs:
    print(active_ingredient_qa_pairs[0][0])
    print(active_ingredient_qa_pairs[0][1])
    print(len(active_ingredient_qa_pairs))

What active ingredients are in drug prescription 19347414?
SELECT ?ActiveIngredient WHERE {
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription/3/patients/19347414> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
        ?Drug <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?ActiveIngredient .
    }
22094


In [100]:
for row in g.query(active_ingredient_qa_pairs[5][1]):
    print([str(item) for item in row])

['https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription/12/patients/15065147/Substance']


# Complex Questions

## Question 5

In [99]:
# Generate question and SPARQL query pairs for the template:
# "Which patients, with the same lab results, have different diagnoses"

same_lab_diff_diag_qa_pairs = []

question = "Which patients, with the same lab results, have different diagnoses?"
sparql_query = """
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult1 .
    ?LabResult1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult2 .
    ?LabResult2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode2 .

    FILTER(?Patient1 != ?Patient2) .
    FILTER(?LabResult1 = ?LabResult2) .
    FILTER(?DiagnosisCode1 != ?DiagnosisCode2) .
}
"""

same_lab_diff_diag_qa_pairs.append((question, sparql_query.strip()))
print(same_lab_diff_diag_qa_pairs[0][0])
print(same_lab_diff_diag_qa_pairs[0][1])

Which patients, with the same lab results, have different diagnoses?
SELECT DISTINCT ?Patient1 ?Patient2 WHERE {
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?LabTestEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult1 .
    ?LabResult1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?LabTestEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasLabTest> ?LabResult2 .
    ?LabResult2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-sc

## Question 6

In [ ]:

diff_diag_qa_pairs = []
question = "What active ingredients are prescribed to patients with different diagnoses from one another?"
sparql_query = """
SELECT DISTINCT ?ActiveIngredient WHERE {
    ?Patient1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    ?Patient2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    FILTER(?Patient1 != ?Patient2) .

    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode2 .

    FILTER(?DiagnosisCode1 != ?DiagnosisCode2) .

    ?DrugPrescription1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DrugPrescription1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/DrugPrescription> .
    ?DrugPrescription1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug1 .
    ?Drug1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasActiveIngredient> ?ActiveIngredient .
}
"""

diff_diag_qa_pairs.append((question, sparql_query.strip()))
print(diff_diag_qa_pairs[0][0])
print(diff_diag_qa_pairs[0][1])



hello
What active ingredients are prescribed to patients with different diagnoses from one another?
SELECT DISTINCT ?ActiveIngredient WHERE {
    ?Patient1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    ?Patient2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/SubjectPseudoIdentifier> .
    FILTER(?Patient1 != ?Patient2) .

    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient1 .
    ?DiagnosisEvent1 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent1 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode1 .

    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Patient2 .
    ?DiagnosisEvent2 a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
    ?DiagnosisEvent2 <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?DiagnosisCode2 .

    FILTER(?DiagnosisCode1 != ?DiagnosisCode2) .

    ?DrugPrescription1 <

In [ ]:
for row in g.query(diff_diag_qa_pairs[0][1]):
    print([str(item) for item in row])

# Medium Comp Qns

## Question 7

In [105]:
procedure_qa_pairs = []
for s, p, o in g.triples((None, URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), URIRef('https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure'))):
    procedure_id = str(s).split('/')[-1]
    question = f"What subjects, Code and Coding System Version does medical procedure {procedure_id} have?"
    sparql_query = f"""
    SELECT ?Subject ?Code ?CodingSystemVersion WHERE {{
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Subject .
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .
        <{s}> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion> ?CodingSystemVersion .
    }}
    """
    procedure_qa_pairs.append((question, sparql_query.strip()))

print(len(procedure_qa_pairs))
if procedure_qa_pairs:
    print(procedure_qa_pairs[0][0])
    print(procedure_qa_pairs[0][1])

1323
What subjects, Code and Coding System Version does medical procedure 19303101 have?
SELECT ?Subject ?Code ?CodingSystemVersion WHERE {
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure/3/patients/19303101> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> ?Subject .
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure/3/patients/19303101> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCode> ?Code .
        <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure/3/patients/19303101> <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasCodingSystemAndVersion> ?CodingSystemVersion .
    }


In [106]:
for row in g.query(procedure_qa_pairs[0][1]):
    print([str(item) for item in row])

['https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/19303101', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/icd#03BC4ZZ', 'https://www.biomedit.ch/rdf/sphn-schema/sphn/v10']


## Question 8

In [107]:
patient_question_query_pairs = []

for s, p, o in g.triples((None, None, None)):
    if 'PATIENTS' in str(s):
        patient_id = str(s).split('/')[-1]
        if patient_id.isdigit():
            question = f"What drugs, diagnosis, lab results and medical procedures did patient {patient_id} have?"
            sparql_query = f"""
            SELECT ?Drug ?Diagnosis ?LabResult ?MedicalProcedure WHERE {{
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
                ?Drug a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Drug> .
                
                ?Diagnosis <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?Diagnosis a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .
                
                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?LabResult a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .
                
                ?MedicalProcedure <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <{s}> .
                ?MedicalProcedure a <https://www.biomedit.ch/rdf/sphn-schema/sphn/MedicalProcedure> .
            }}
            """
            patient_question_query_pairs.append((question, sparql_query.strip()))

print(len(patient_question_query_pairs))
if patient_question_query_pairs:
    print(patient_question_query_pairs[0][0])
    print(patient_question_query_pairs[0][1])

790
What drugs, diagnosis, lab results and medical procedures did patient 16230249 have?
SELECT ?Drug ?Diagnosis ?LabResult ?MedicalProcedure WHERE {
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/16230249> .
                ?DrugPres <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasDrug> ?Drug .
                ?Drug a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Drug> .

                ?Diagnosis <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/16230249> .
                ?Diagnosis a <https://www.biomedit.ch/rdf/sphn-schema/sphn/Diagnosis> .

                ?LabResult <https://www.biomedit.ch/rdf/sphn-schema/sphn/hasSubjectPseudoIdentifier> <https://www.biomedit.ch/rdf/sphn-schema/sphn/PATIENTS/16230249> .
                ?LabResult a <https://www.biomedit.ch/rdf/sphn-schema/sphn/LabResult> .

 

In [108]:
for row in g.query(patient_question_query_pairs[0][1]):
    print([str(item) for item in row])

KeyboardInterrupt: 